In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-sample.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-labels.pkl

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import malaya_speech
import json
import pickle
from tqdm import tqdm
import tensorflow as tf
import malaya_speech.train.model.conformer as conformer

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [4]:
with open('voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [5]:
with open('voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [6]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))

In [7]:
import torch

wav2mel = torch.jit.load('wav2mel.pt')
dvector = torch.jit.load('dvector-step250000.pt').eval()

In [8]:
# _ = wav2mel.cuda()
_ = dvector.cuda()

In [9]:
vectors = {}
for f in tqdm(unique_files):
    y_, _ = malaya_speech.load(f.replace('/home/husein/youtube/', '/home/husein/'))
    try:
        mel_tensor = wav2mel(torch.tensor(y_.astype('float32')).unsqueeze(0), 16000)
        emb_tensor = dvector.embed_utterance(mel_tensor.cuda())
        v = emb_tensor.cpu().detach().numpy()
        vectors[f] = v
        
    except Exception as e:
        print(e)
        torch.cuda.empty_cache()

  0%|                                                 | 0/36237 [00:00<?, ?it/s]/tmp/ipykernel_3651179/3707568404.py:6: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
  emb_tensor = dvector.embed_utterance(mel_tensor.cuda())
100%|█████████████████████████████████████| 36237/36237 [07:39<00:00, 78.82it/s]


In [14]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    if labels[i][1] in vectors and labels[i][2] in vectors:
        ls.append(labels[i][0])
        scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|█████████████████████████████| 5900000/5900000 [00:15<00:00, 380148.90it/s]


In [15]:
len(scores), len(vectors)

(5900000, 36237)

In [16]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [17]:
calculate_eer(ls, scores)

(0.13564905982985956, array(0.20760974))